In [30]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys, time, glob
import pyccl as ccl
import matplotlib.pyplot as plt
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]
from astropy.table import Table
sys.path.append('/pbs/throng/lsst/users/cpayerne/LikelihoodsClusterAbundance/modules/')
import importance_sampling as Oms8
import abundance as cl_count
import covariance as covar
import mvp_pdf
import edit
import utils
from lnlikelihood import lnLikelihood

In [31]:
n_z_bin=100
n_logm_bin=100
ratio_f_sky=1
label= 'sup_5e14Msun'
dat=edit.load_pickle(f'/sps/lsst/users/cpayerne/1000xsimulations/analysis/binned_catalogs/{n_z_bin}x{n_logm_bin}_binned_catalogs_'+label+'.pkl')
Nobs = dat[3]
#Covariances
where_covar='/pbs/throng/lsst/users/cpayerne/ClusterLikelihoods_data/Covariances/'
S_ij = edit.load_pickle(where_covar+label+f'_Sij_partialsky_blockdiagonal_{n_z_bin}x{n_logm_bin}.pickle')
full_covariance = edit.load_pickle(where_covar+label+f'_Covariance_cluster_abudance_{n_z_bin}x{n_logm_bin}.pickle') 
Cholesky=np.linalg.cholesky(full_covariance * ratio_f_sky)
inv_L = np.linalg.inv(Cholesky)
inv_full_cov = np.linalg.inv(full_covariance)
Halo_bias=edit.load_pickle(where_covar+label+f'_Halo_bias_{n_z_bin}x{n_logm_bin}.pickle') 
theory={'S_ij':(1./ratio_f_sky)*S_ij, 
        'inv_full_covariance': (1./ratio_f_sky)*inv_full_cov, 
        'Halo_bias':Halo_bias, 
        'inv_L': inv_L}

#tabulated_model-in a single array
where_tab=f'/sps/lsst/users/cpayerne/1000xsimulations/analysis/'
key=f'{n_z_bin}zx{n_logm_bin}m/tabulated_model'+'_'+label+f'/{n_z_bin}x{n_logm_bin}_sampled_abundance_' 
where=where_tab+key+'*'
Nth, Om, s8, q_val=[],[],[],[]
file=glob.glob(where)

In [32]:
for f in np.array(file)[10:20]:
    sampled_model=np.array(edit.load_pickle(f))
    q=sampled_model['q']
    Nth.extend(ratio_f_sky * sampled_model['abundance'])
    q_val.extend(list(sampled_model['q']))
    Om.extend(list(sampled_model['Om']))
    s8.extend(list(sampled_model['s8']))

In [47]:
likelihood = 'MPG'

In [48]:
lnL=lnLikelihood(theory=theory)
def lnposterior(model, data):
    #define the posterior
    return lnL.lnPosterior(model, data, likelihood=likelihood)

t = Table()
def iter(n): return Oms8.compute_mean_covariance_importance_sampling(lnposterior, Nobs[n], Nth[0:1], 
                                                                     Om=Om[0:1], s8=s8[0:1], 
                                                                     q_val=q_val[0:1], 
                                                                     mp=False, browse=False)
print('computing')

computing
[-5834.02530769]
0


array([0.2689459600953529, 0.8311071463685029, array([[nan, nan],
                                                      [nan, nan]])],
      dtype=object)

In [50]:
import time
ti = time.time()
iter(0)
tf = time.time()
print(tf-ti)

[-5834.02530769]
0
5.034759044647217


In [29]:
iter(0)

100


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [7]:
mask = q_val_==None

NameError: name 'q_val_' is not defined

In [ ]:
mask

In [ ]:
res = []
n_max = 199
for i, nth in enumerate(Nth):
    if i > n_max: break
    res.append(lnposterior(nth, dat[3][0]))

In [ ]:
plt.hist(res)

In [ ]:
q_val_ = np.array(q_val)[np.arange(n_max+1)]

In [ ]:
q_val_

In [ ]:
plt.hist(q_val)

In [ ]:
len(q_val)

In [ ]:
res = np.array(res)

In [ ]:
res1 = res-np.median(res)

In [ ]:
res_ratio = res - np.log(q_val_)

In [ ]:
res_ratio_norm = res_ratio - np.median(res_ratio)

In [ ]:
res_ratio_norm